In [1]:
import numpy as np
import itertools
import pandas as pd
import json

from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve, _SigmoidCalibration
from ampligraph.evaluation import evaluate_performance, mr_score, mrr_score, hits_at_n_score, generate_corruptions_for_eval
from sklearn.metrics import brier_score_loss, log_loss, accuracy_score
from scipy.special import expit

from ampligraph.datasets import load_yago39k
from ampligraph.latent_features.models import TransE, ComplEx, DistMult, ConvKB, HolE
import types

In [2]:
from generate_corruptions import generate_corruptions, calibration_loss, pos_iso

In [3]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [4]:
X = load_yago39k()

In [5]:
X_valid_pos = X['valid'][X['valid_labels']]
X_valid_neg = X['valid'][~X['valid_labels']]

X_test_pos = X['test'][X['test_labels']]
X_test_neg = X['test'][~X['test_labels']]

In [6]:
losses =  ['self_adversarial', 'pairwise', 'nll', 'multiclass_nll']
models = [TransE, DistMult, ComplEx, HolE]

results = []

for m, l in itertools.product(models, losses):
    model = m(batches_count=64, seed=0, epochs=1000, k=100, eta=20,
                   optimizer='adam', optimizer_params={'lr':0.0001},
                   loss=l, verbose=False)

    model.fit(X['train'])
    
    scores = model.predict(X['test'])

    model.calibrate(X_valid_pos, batches_count=10, epochs=1000, positive_base_rate=0.5)
    print("pos", model.calibration_parameters)
    probas1 = model.predict_proba(X['test'])

    model.calibrate(X_valid_pos, X_valid_neg)
    print("pos neg", model.calibration_parameters)
    probas2 = model.predict_proba(X['test'])
    
    val_scores = model.predict(X['valid'])
    ir = IsotonicRegression(out_of_bounds='clip')
    ir.fit(np.squeeze(val_scores).astype(float), (X['valid_labels']).astype(float))
    probas3 = ir.predict(np.squeeze(scores).astype(float))
    
    model.generate_corruptions = types.MethodType(generate_corruptions, model)
    corruptions = model.generate_corruptions(X_valid_pos, batches_count=10, epochs=1000)
    val_pos_scores = np.squeeze(model.predict(X_valid_pos))
    iso_pos = pos_iso(IsotonicRegression(out_of_bounds='clip'), val_pos_scores, corruptions, positive_base_rate=0.5)
    probas4 = iso_pos.predict(np.squeeze(scores).astype(float))

    sc_pos = pos_iso(_SigmoidCalibration(), val_pos_scores, corruptions, positive_base_rate=0.5)
    print("pos sc", sc_pos.a_, sc_pos.b_)
    probas5 = sc_pos.predict(np.squeeze(scores).astype(float))
    
    val_neg_scores = np.squeeze(model.predict(X_valid_neg))
    sc_pos_neg = pos_iso(_SigmoidCalibration(), val_pos_scores, val_neg_scores, positive_base_rate=0.5)
    print("pos neg sc", sc_pos_neg.a_, sc_pos_neg.b_)
    probas6 = sc_pos_neg.predict(np.squeeze(scores).astype(float))
    
    thresholds = {r: np.median(np.sort(val_scores[X['valid'][:, 1] == r])) for r in np.unique(X['valid'][:, 1])}
    thresholds_test = np.vectorize(thresholds.get)(X['test'][:, 1])
    per_relation_acc = accuracy_score(X['test_labels'], scores > thresholds_test)

    acc_uncalib = accuracy_score(X['test_labels'], expit(scores) > 0.5)

    acc1 = accuracy_score(X['test_labels'], probas1 > 0.5)
    acc2 = accuracy_score(X['test_labels'], probas2 > 0.5)
    acc3 = accuracy_score(X['test_labels'], probas3 > 0.5)
    acc4 = accuracy_score(X['test_labels'], probas4 > 0.5)
    acc5 = accuracy_score(X['test_labels'], probas5 > 0.5)
    acc6 = accuracy_score(X['test_labels'], probas6 > 0.5)
    
    filter_triples = np.concatenate((X['train'], X_valid_pos, X_test_pos))
    ranks = evaluate_performance(X_test_pos, 
                                 model=model, 
                                 filter_triples=filter_triples,
                                 use_default_protocol=True, 
                                 verbose=False)

    results.append({
        'model': m.__name__,
        'loss': l,
        'brier_score_scores': brier_score_loss(X['test_labels'], expit(scores)),
        'log_loss_scores': log_loss(X['test_labels'], expit(scores), eps=1e-7),
        'brier_score_probas_pos': brier_score_loss(X['test_labels'], probas1),
        'log_loss_probas_pos': log_loss(X['test_labels'], probas1, eps=1e-7),
        'brier_score_probas_pos_neg': brier_score_loss(X['test_labels'], probas2),
        'log_loss_probas_pos_neg': log_loss(X['test_labels'], probas2, eps=1e-7),
        'brier_score_probas_pos_neg_iso': brier_score_loss(X['test_labels'], probas3),
        'log_loss_probas_pos_neg_iso': log_loss(X['test_labels'], probas3, eps=1e-7),
        'brier_score_probas_pos_iso': brier_score_loss(X['test_labels'], probas4),
        'log_loss_probas_pos_iso': log_loss(X['test_labels'], probas4, eps=1e-7),
        'brier_score_probas_pos_sc': brier_score_loss(X['test_labels'], probas5),
        'log_loss_probas_pos_sc': log_loss(X['test_labels'], probas5, eps=1e-7),
        'brier_score_probas_pos_neg_sc': brier_score_loss(X['test_labels'], probas6),
        'log_loss_probas_pos_neg_sc': log_loss(X['test_labels'], probas6, eps=1e-7),
        'ece_scores': calibration_loss(X['test_labels'], expit(scores)),
        'ece_probas_pos': calibration_loss(X['test_labels'], probas1),
        'ece_probas_pos_neg': calibration_loss(X['test_labels'], probas2),
        'ece_probas_pos_neg_iso': calibration_loss(X['test_labels'], probas3),
        'ece_probas_pos_iso': calibration_loss(X['test_labels'], probas4),
        'ece_probas_pos_sc': calibration_loss(X['test_labels'], probas5),
        'ece_probas_pos_neg_sc': calibration_loss(X['test_labels'], probas6),
        'metrics_mrr': mrr_score(ranks), 
        'metrics_hits@10': hits_at_n_score(ranks, n=10),
        'metrics_mr': mr_score(ranks),
        'accuracy_per_relation': per_relation_acc,
        'accuracy_uncalib': acc_uncalib,
        'accuracy_pos': acc1,
        'accuracy_pos_neg': acc2,
        'accuracy_pos_neg_iso': acc3,
        'accuracy_pos_iso': acc4,
        'accuracy_pos_sc': acc5,
        'accuracy_pos_neg_sc': acc6
    })
        
    print(json.dumps(results[-1], indent=2))

WARNING - From /home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
WARNING - From /home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:429: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
WARNING - From /home/ptabacof/.

/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in true_divide
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count


{
  "model": "TransE",
  "loss": "self_adversarial",
  "brier_score_scores": 0.3631659630424295,
  "log_loss_scores": 1.0628101241289667,
  "brier_score_probas_pos": 0.11326054212735774,
  "log_loss_probas_pos": 0.39898948892097613,
  "brier_score_probas_pos_neg": 0.09484812404036443,
  "log_loss_probas_pos_neg": 0.31861535438327826,
  "brier_score_probas_pos_neg_iso": 0.09253307079167732,
  "log_loss_probas_pos_neg_iso": 0.30813644217075253,
  "brier_score_probas_pos_iso": 0.10932302495335325,
  "log_loss_probas_pos_iso": 0.3755526695921677,
  "brier_score_probas_pos_sc": 0.1061865371173744,
  "log_loss_probas_pos_sc": 0.3880795680196811,
  "brier_score_probas_pos_neg_sc": 0.09480374936218143,
  "log_loss_probas_pos_neg_sc": 0.31861897355361735,
  "ece_scores": 0.4083999011337116,
  "ece_probas_pos": 0.11869738999233456,
  "ece_probas_pos_neg": 0.02867179460422967,
  "ece_probas_pos_neg_iso": 1.3722595342006639e-17,
  "ece_probas_pos_iso": 0.09610577860440293,
  "ece_probas_pos_sc": 0

/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in true_divide
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count


{
  "model": "TransE",
  "loss": "pairwise",
  "brier_score_scores": 0.49772074388347165,
  "log_loss_scores": 4.921945735433863,
  "brier_score_probas_pos": 0.1470969047874042,
  "log_loss_probas_pos": 0.47739234029725996,
  "brier_score_probas_pos_neg": 0.123208328346179,
  "log_loss_probas_pos_neg": 0.44454980222833146,
  "brier_score_probas_pos_neg_iso": 0.10320889810531554,
  "log_loss_probas_pos_neg_iso": 0.3517519332316571,
  "brier_score_probas_pos_iso": 0.11293827956899179,
  "log_loss_probas_pos_iso": 0.39293455449606485,
  "brier_score_probas_pos_sc": 0.13953547628203666,
  "log_loss_probas_pos_sc": 0.4784827552125384,
  "brier_score_probas_pos_neg_sc": 0.12332184709502418,
  "log_loss_probas_pos_neg_sc": 0.4445549083746815,
  "ece_scores": 0.49774829798488207,
  "ece_probas_pos": 0.19326106799546108,
  "ece_probas_pos_neg": 0.14001669688891338,
  "ece_probas_pos_neg_iso": 9.060852426779504e-17,
  "ece_probas_pos_iso": 0.06509857470870574,
  "ece_probas_pos_sc": 0.1328616564

...

pos sc -0.2902491500954467 2.875854107744645
pos neg sc -0.26466027386028973 3.4546922119320405
{
  "model": "ComplEx",
  "loss": "multiclass_nll",
  "brier_score_scores": 0.42882664237150747,
  "log_loss_scores": 3.3327318642598676,
  "brier_score_probas_pos": 0.1512251639874778,
  "log_loss_probas_pos": 0.4631039282367677,
  "brier_score_probas_pos_neg": 0.10790711784846009,
  "log_loss_probas_pos_neg": 0.3535279664224244,
  "brier_score_probas_pos_neg_iso": 0.10117277542640077,
  "log_loss_probas_pos_neg_iso": 0.3167186081062865,
  "brier_score_probas_pos_iso": 0.12303168877636766,
  "log_loss_probas_pos_iso": 0.3718943251864612,
  "brier_score_probas_pos_sc": 0.12903198548485248,
  "log_loss_probas_pos_sc": 0.3963750720348282,
  "brier_score_probas_pos_neg_sc": 0.10799402411755174,
  "log_loss_probas_pos_neg_sc": 0.35353239354011284,
  "ece_scores": 0.4444651029443228,
  "ece_probas_pos": 0.1425259727560064,
  "ece_probas_pos_neg": 0.07070543022565945,
  "ece_probas_pos_neg_iso": 1

/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in true_divide
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count


{
  "model": "HolE",
  "loss": "self_adversarial",
  "brier_score_scores": 0.3447489880655823,
  "log_loss_scores": 1.064809434521654,
  "brier_score_probas_pos": 0.16595915658987734,
  "log_loss_probas_pos": 0.5809735449744614,
  "brier_score_probas_pos_neg": 0.14122897401633186,
  "log_loss_probas_pos_neg": 0.4444461881074887,
  "brier_score_probas_pos_neg_iso": 0.1397822585690421,
  "log_loss_probas_pos_neg_iso": 0.4378594972058075,
  "brier_score_probas_pos_iso": 0.16151406433428497,
  "log_loss_probas_pos_iso": 0.5374365597492345,
  "brier_score_probas_pos_sc": 0.15915575685251362,
  "log_loss_probas_pos_sc": 0.5362728578401544,
  "brier_score_probas_pos_neg_sc": 0.14122142220729475,
  "log_loss_probas_pos_neg_sc": 0.4444516582881186,
  "ece_scores": 0.38560502218943776,
  "ece_probas_pos": 0.13877935265982022,
  "ece_probas_pos_neg": 0.016449740009923136,
  "ece_probas_pos_neg_iso": 1.0183759451013952e-17,
  "ece_probas_pos_iso": 0.12271153560737806,
  "ece_probas_pos_sc": 0.1205

/home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/sklearn/calibration.py:453: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/sklearn/calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)


pos sc -0.47738997078888845 3.5277458004878306
pos neg sc -0.43882475900274465 4.236763150984862
{
  "model": "HolE",
  "loss": "multiclass_nll",
  "brier_score_scores": 0.4253507254620633,
  "log_loss_scores": 2.7565118307325593,
  "brier_score_probas_pos": 0.1544277223558572,
  "log_loss_probas_pos": 0.49349217955077895,
  "brier_score_probas_pos_neg": 0.10973199893689896,
  "log_loss_probas_pos_neg": 0.3623903886565794,
  "brier_score_probas_pos_neg_iso": 0.1043751118995447,
  "log_loss_probas_pos_neg_iso": 0.3438977020464111,
  "brier_score_probas_pos_iso": 0.1291009404550466,
  "log_loss_probas_pos_iso": 0.41379817176355704,
  "brier_score_probas_pos_sc": 0.1349553628876394,
  "log_loss_probas_pos_sc": 0.42455919865883557,
  "brier_score_probas_pos_neg_sc": 0.10980081442350165,
  "log_loss_probas_pos_neg_sc": 0.36239485402866195,
  "ece_scores": 0.44808953821018177,
  "ece_probas_pos": 0.15103253682454426,
  "ece_probas_pos_neg": 0.05021793734642767,
  "ece_probas_pos_neg_iso": 2.

In [7]:
import pandas as pd

In [8]:
def highlight_min(s):
    is_min = s == s.min()
    return ['font-weight: bold' if v else '' for v in is_min]

In [18]:
df = pd.DataFrame(results).set_index(['model', 'loss'])
df.to_csv("main_results_yago39k.csv", index=False)

In [10]:
bs = df[(c for c in df.columns if c.startswith('brier'))]
bs.columns = [c[len("brier_score_"):] for c in bs.columns]
bs.style.apply(highlight_min, axis=1)

In [11]:
ll = df[(c for c in df.columns if c.startswith('log_loss'))]
ll.columns = [c[len("log_loss_"):] for c in ll.columns]
ll.style.apply(highlight_min, axis=1)

In [12]:
print((bs.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'scores', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
 .round(3)
 .to_latex()))

\begin{tabular}{llrrrrr}
\toprule
{} &     model &  scores &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &    TransE &   0.363 &           0.095 &               0.093 &       0.113 &           0.109 \\
1 &  DistMult &   0.284 &           0.081 &               0.079 &       0.091 &           0.089 \\
2 &   ComplEx &   0.264 &           0.089 &               0.084 &       0.095 &           0.095 \\
3 &      HolE &   0.345 &           0.141 &               0.140 &       0.166 &           0.162 \\
\bottomrule
\end{tabular}



In [13]:
print(ll.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'scores', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrrr}
\toprule
{} &     model &  scores &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &    TransE &   1.063 &           0.319 &               0.308 &       0.399 &           0.376 \\
1 &  DistMult &   1.043 &           0.279 &               0.266 &       0.309 &           0.308 \\
2 &   ComplEx &   1.199 &           0.305 &               0.278 &       0.319 &           0.313 \\
3 &      HolE &   1.065 &           0.444 &               0.438 &       0.581 &           0.537 \\
\bottomrule
\end{tabular}



In [15]:
print(bs.reset_index()
 .query("model == 'TransE' ")
 [['loss',  'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrr}
\toprule
{} &              loss &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &  self\_adversarial &           0.095 &               0.093 &       0.113 &           0.109 \\
1 &          pairwise &           0.123 &               0.103 &       0.147 &           0.113 \\
2 &               nll &           0.187 &               0.170 &       0.260 &           0.200 \\
3 &    multiclass\_nll &           0.111 &               0.104 &       0.128 &           0.116 \\
\bottomrule
\end{tabular}



In [16]:
print(ll.reset_index()
 .query("model == 'TransE' ")
 [['loss', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrr}
\toprule
{} &              loss &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &  self\_adversarial &           0.319 &               0.308 &       0.399 &           0.376 \\
1 &          pairwise &           0.445 &               0.352 &       0.477 &           0.393 \\
2 &               nll &           0.577 &               0.518 &       0.756 &           0.622 \\
3 &    multiclass\_nll &           0.392 &               0.350 &       0.431 &           0.440 \\
\bottomrule
\end{tabular}



In [23]:
acc = df[(c for c in df.columns if c.startswith('accuracy'))]
acc.columns = [c[len("accuracy_"):] for c in acc.columns]
acc.style.apply(highlight_min, axis=1)

In [24]:
print((acc*100).reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'pos_neg', 'pos_neg_iso', 'pos', 'pos_iso',  'uncalib', 'per_relation']]
 .reset_index(drop=True)
  .round(1)
 .to_latex())

\begin{tabular}{llrrrrrr}
\toprule
{} &     model &  pos\_neg &  pos\_neg\_iso &   pos &  pos\_iso &  uncalib &  per\_relation \\
\midrule
0 &    TransE &     87.2 &         87.8 &  85.3 &     84.9 &     50.2 &          88.8 \\
1 &  DistMult &     88.9 &         89.3 &  88.1 &     88.5 &     56.7 &          90.2 \\
2 &   ComplEx &     87.3 &         88.2 &  86.9 &     87.2 &     61.1 &          89.4 \\
3 &      HolE &     80.4 &         80.4 &  78.4 &     78.5 &     50.6 &          81.5 \\
\bottomrule
\end{tabular}



In [17]:
metrics = df[(c for c in df.columns if c.startswith('metrics'))]
metrics.columns = [c[len("metrics_"):] for c in metrics.columns]
metrics

hits@10           mr       mrr
model    loss                                             
TransE   self_adversarial  0.319365   244.245221  0.168552
         pairwise          0.648666   500.205260  0.371433
         nll               0.113187  1012.296846  0.063915
         multiclass_nll    0.590021   216.366508  0.324910
DistMult self_adversarial  0.619883   635.099039  0.305570
         pairwise          0.633060   648.001242  0.337370
         nll               0.647262   744.198078  0.330295
         multiclass_nll    0.585268   397.900475  0.308723
ComplEx  self_adversarial  0.752673  1074.095583  0.530927
         pairwise          0.729182   587.079436  0.476633
         nll               0.731882   855.117021  0.493711
         multiclass_nll    0.747165   326.583432  0.475104
HolE     self_adversarial  0.188519   921.523005  0.101279
         pairwise          0.292472   394.971649  0.146192
         nll               0.235285   728.628254  0.111022
         multiclass_nll    0.299816   343.793390  0.157926

In [22]:
print(metrics.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'mr', 'mrr', 'hits@10']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrr}
\toprule
{} &     model &        mr &    mrr &  hits@10 \\
\midrule
0 &    TransE &   244.245 &  0.169 &    0.319 \\
1 &  DistMult &   635.099 &  0.306 &    0.620 \\
2 &   ComplEx &  1074.096 &  0.531 &    0.753 \\
3 &      HolE &   921.523 &  0.101 &    0.189 \\
\bottomrule
\end{tabular}



In [ ]:
def highlight_max(s):
    is_min = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_min]

acc = df[(c for c in df.columns if c.startswith('accuracy'))]
acc.columns = [c[len("accuracy_"):] for c in acc.columns]
acc.style.apply(highlight_max, axis=1)

In [ ]:
df.corr(method='spearman')

In [ ]:
len(np.unique(X['valid'][:, 1]))

In [19]:
model = TransE(batches_count=64, seed=0, epochs=1000, k=100, eta=20,
               optimizer='adam', optimizer_params={'lr':0.0001},
               loss='self_adversarial', verbose=False)

model.fit(X['train'])

scores = model.predict(X['test'])

model.calibrate(X_valid_pos, batches_count=10, epochs=1000, positive_base_rate=0.5)
print("pos", model.calibration_parameters)
probas1 = model.predict_proba(X['test'])

model.calibrate(X_valid_pos, X_valid_neg)
print("pos neg", model.calibration_parameters)
probas2 = model.predict_proba(X['test'])

val_scores = model.predict(X['valid'])

thresholds = {r: np.median(np.sort(val_scores[X['valid'][:, 1] == r])) for r in np.unique(X['valid'][:, 1])}
thresholds_test = np.vectorize(thresholds.get)(X['test'][:, 1])
per_relation_acc = accuracy_score(X['test_labels'], scores > thresholds_test)

print(thresholds)


pos [-2.0008023, -7.3683147]
pos neg [-1.8528421, -5.587557]
{0: -3.9869666, 1: -3.6161883, 2: -2.9660778, 3: -2.9241138, 4: -3.8640308, 5: -3.685308, 6: -2.861393, 7: -3.3280334, 8: -3.0741293, 9: -3.1950998, 10: -2.951118, 11: -1.8720441, 12: -2.4230814, 13: -1.542841, 14: -2.6944544, 15: -3.381497, 16: -1.8443029, 17: -3.4323683, 18: -1.6325312, 19: -4.2211304, 20: -4.101904, 21: -3.840962, 22: -1.832546, 23: -2.0101485, 24: -3.1512089, 25: -2.4524217, 27: -3.4848583, 29: -2.4269128, 31: -2.209188, 32: -1.3310984, 33: -2.3231838, 35: -2.0017974, 36: -1.3954651}


In [20]:
per_relation_acc

0.8876881720430108